In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.

In [ ]:
##db = sqlite3.connect('data/doc_grahp_hop.sqlite')

##chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
##hop = []
##for chunk in chunks: 
##    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
##    hop.append(chunk)
##    chunk.to_sql('hop', db, if_exists = 'append', index = False) 

In [ ]:
##hopp = pd.concat(hop, ignore_index = True)
##hopp


In [2]:
cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code', 'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']

nppes_chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', usecols=cols, iterator=True, chunksize=1000)

In [3]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.concat([chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)] for chunk in nppes_chunks])

In [4]:
nppes_l = pd.melt(nppes)
nppes_l

,variable,value
0,NPI,1750384210
1,NPI,1386647840
2,NPI,1922001486
3,NPI,1992708796
4,NPI,1760485551
...,...,...
28630399,Healthcare Provider Primary Taxonomy Switch_15,NaN
28630400,Healthcare Provider Primary Taxonomy Switch_15,NaN
28630401,Healthcare Provider Primary Taxonomy Switch_15,NaN
28630402,Healthcare Provider Primary Taxonomy Switch_15,NaN


In [6]:
nppes_l.loc[nppes_l['Healthcare Provider Primary Taxonomy Switch_{i}'] = 'Y'] = nppes_l['Healthcare Provider Primary Taxonomy Switch_{i}'] = 'N';

SyntaxError: invalid syntax (2799361150.py, line 1)

In [ ]:
taxonomy = pd.read_csv('data/nucc_taxonomy_230.csv', nrows=100)
taxonomy

In [ ]:
##db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

##chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023.csv', chunksize = 10000)
##nppes = []
##for chunk in chunks: 

##    hop.append(chunk)